In [ ]:
from myria import *
import numpy

# Load Myria extensions
%load_ext myria

# Create Jupyter Connection
%connect http://localhost:8753 http://localhost:8080

In [ ]:
%%query
T1 = scan(s1_seq);
s_sub = select * from T1 limit 100;
store(s_sub, s1_seq_sub);

In [ ]:
%%query
T2 = scan(s2_seq);
s_sub = select * from T2 limit 100;
store(s_sub, s2_seq_sub);

In [ ]:
%%query
T1 = scan(s1_seq_sub);
s = select "s1" as sample, ngram(seq, 11) as kmer from T1;
s_cnt = select sample, kmer, count(kmer) as cnt from s;
store(s_cnt, s1, [kmer]);


In [ ]:
%%query
T1 = scan(s2_seq_sub);
s = select "s2" as sample, ngram(seq, 11) as kmer from T1;
s_cnt = select sample, kmer, count(kmer) as cnt from s;
store(s_cnt, s2, [kmer]);

In [ ]:
%%query
s1 = scan(s1);
s2 = scan(s2);
s1c = select sample, sum(cnt) as sum_cnt from s1;
s2c = select sample, sum(cnt) as sum_cnt from s2;
store(s1c, s1_cnt);
store(s2c, s2_cnt);

In [ ]:
%%query
s1 = scan(s1);
s2 = scan(s2);
s = s1 + s2;
s1c = scan(s1_cnt);
s2c = scan(s2_cnt);
sums = s1c + s2c;
sn = select samps.kmer, samps.sample, samps.cnt/sc.sum_cnt as norm_cnt from s samps, sums sc where samps.sample = sc.sample;
store(sn, s_norms_cnt, [kmer]);


In [ ]:
%%query
kmercnts = scan(s_norms_cnt);
entropy = select k.sample, 22+sum(k.norm_cnt*log(k.norm_cnt)/log(2)) as entropy from kmercnts k;
store(entropy, entropy_combined);


In [ ]:
%%query
kmers = scan(s_norms_cnt);
X = select a.sample as asample, b.sample as bsample,
    case when a.norm_cnt < b.norm_cnt then a.norm_cnt else b.norm_cnt end as minv
    from kmers a, kmers b
    where a.kmer = b.kmer and a.sample < b.sample;
Y = select asample, bsample, 1 - sum(minv) as BCdis from X;
store(Y, BC_s1_s2);
